In [2]:
import numpy as np
import tensorflow as tf
import pydicom
import os
from skimage import measure, morphology
from scipy.ndimage import zoom
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Función para crear un volumen 3D desde múltiples archivos DICOM
def create_3d_volume(dicom_files):
    """
    Crea un volumen 3D a partir de múltiples cortes DICOM.
    Args:
        dicom_files: Lista de rutas a archivos DICOM.
    Returns:
        volume: Volumen 3D en valores HU.
        spacing: Resolución espacial del volumen (z, y, x).
    """
    slices = []
    spacings = []

    for dicom_path in dicom_files:
        try:
            dicom = pydicom.dcmread(dicom_path)
            image = dicom.pixel_array.astype(np.int16)

            # Normalizar valores HU
            image[image == -2000] = 0
            intercept = dicom.RescaleIntercept
            slope = dicom.RescaleSlope
            image = image * slope + intercept
            slices.append(image)

            # Extraer la resolución del voxel
            pixel_spacing = getattr(dicom, "PixelSpacing", [1, 1])
            slice_thickness = getattr(dicom, "SliceThickness", 1)
            spacings.append(list(pixel_spacing) + [slice_thickness])

        except PermissionError:
            print(f"No se pudo acceder al archivo: {dicom_path}. Verifica permisos.")
            continue
        except Exception as e:
            print(f"Error al procesar el archivo {dicom_path}: {e}")
            continue

    # Ordenar los cortes por su posición en Z (si hay un atributo SliceLocation)
    slices = np.array(slices)
    spacings = np.mean(spacings, axis=0) if spacings else [1, 1, 1]  # Espaciado por defecto si no se encuentra
    return np.stack(slices, axis=0), spacings

# Función para segmentar los pulmones en un volumen 3D
def segment_lungs(volume):
    """
    Segmenta los pulmones en un volumen 3D basado en valores HU.
    Args:
        volume: Volumen 3D en valores HU.
    Returns:
        segmented_volume: Volumen segmentado.
        mask: Máscara binaria de los pulmones.
    """
    binary_mask = (volume > -700) & (volume < -600)
    binary_mask = morphology.remove_small_objects(binary_mask, min_size=500)

    # Etiquetar regiones conectadas
    labeled_mask = measure.label(binary_mask, connectivity=1)
    mask = morphology.remove_small_objects(labeled_mask, min_size=1000)

    # Aplicar la máscara al volumen original
    segmented_volume = volume * (mask > 0)
    return segmented_volume, mask

# Función para dividir el volumen en bloques
def block_volume(segmented_volume, block_size=(64, 64, 64)):
    """
    Divide el volumen segmentado en bloques más pequeños.
    Args:
        segmented_volume: Volumen segmentado.
        block_size: Tamaño de cada bloque (profundidad, alto, ancho).
    Returns:
        Lista de bloques extraídos.
    """
    d, h, w = segmented_volume.shape
    blocks = []
    for z in range(0, d, block_size[0]):
        for y in range(0, h, block_size[1]):
            for x in range(0, w, block_size[2]):
                block = segmented_volume[z:z + block_size[0], y:y + block_size[1], x:x + block_size[2]]
                if block.shape == block_size:
                    blocks.append(block)
    return np.array(blocks)



In [ ]:
# Construcción de modelo CNN 3D para clasificación binaria
def build_cnn_3d(input_shape=(64, 64, 64, 1)):
    """
    Construye un modelo CNN 3D para clasificación de imágenes en 2 clases (fibrosis y cáncer).
    Args:
        input_shape: Forma de entrada (profundidad, alto, ancho, canales).
    Returns:
        Modelo CNN 3D compilado.
    """
    model = tf.keras.Sequential([
        tf.keras.layers.Conv3D(32, (3, 3, 3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling3D((2, 2, 2)),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Conv3D(64, (3, 3, 3), activation='relu'),
        tf.keras.layers.MaxPooling3D((2, 2, 2)),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(2, activation='softmax')  # Dos clases: 0 = FPI, 1 = No FPI
    ])
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Función principal para entrenar el modelo
def main():
    root_dir = "C:/Users/Marcos/reto-1-mas-alla-de-la-mirada-humana"  # Cambia esta ruta a tu directorio principal
    fibrosis_dir = os.path.join(root_dir, 'fibrosis_reducido')  # Ruta a la carpeta de fibrosis
    cancer_dir = os.path.join(root_dir, 'CT_cancer')  # Ruta a la carpeta de cáncer

    all_blocks = []
    labels = []  # Etiquetas esperadas: 0 (FPI), 1 (No FPI)

    # Procesar FPI
    for patient_dir in os.listdir(fibrosis_dir):
        patient_path = os.path.join(fibrosis_dir, patient_dir)
        if os.path.isdir(patient_path):
            dicom_files = [os.path.join(patient_path, f) for f in os.listdir(patient_path) if f.endswith('.dcm')]
            volume, spacing = create_3d_volume(dicom_files)
            segmented_volume, mask = segment_lungs(volume)
            blocks = block_volume(segmented_volume)
            all_blocks.extend(blocks)
            labels.extend([0] * len(blocks))  # Etiqueta 0 para FPI

    # Procesar No FPI
    for patient_dir in os.listdir(cancer_dir):
        patient_path = os.path.join(cancer_dir, patient_dir)
        if os.path.isdir(patient_path):
            dicom_files = [os.path.join(patient_path, f) for f in os.listdir(patient_path) if f.endswith('.dcm')]
            volume, spacing = create_3d_volume(dicom_files)
            segmented_volume, mask = segment_lungs(volume)
            blocks = block_volume(segmented_volume)
            all_blocks.extend(blocks)
            labels.extend([1] * len(blocks))  # Etiqueta 1 para No FPI

    all_blocks = np.array(all_blocks)
    all_blocks = all_blocks[..., np.newaxis]  # Agregar dimensión de canal para TensorFlow
    labels = tf.keras.utils.to_categorical(labels, num_classes=2)  # Convertir a formato categórico

    # Dividir datos en entrenamiento y validación
    X_train, X_val, y_train, y_val = train_test_split(all_blocks, labels, test_size=0.2, random_state=42)

    # Entrenar el modelo CNN 3D
    cnn_model = build_cnn_3d(input_shape=X_train.shape[1:])
    history = cnn_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=4, batch_size=4)

    # Evaluar el modelo
    val_loss, val_accuracy = cnn_model.evaluate(X_val, y_val)
    print(f"Precisión en validación: {val_accuracy:.2f}")

    # Guardar el modelo
    cnn_model.save("fibrosis_cancer_model_3d.h5")

if __name__ == "__main__":
    main()


c:\Users\Marcos\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/4
557/557 ━━━━━━━━━━━━━━━━━━━━ 225s 399ms/step - accuracy: 0.6599 - loss: 2.4854 - val_accuracy: 0.6894 - val_loss: 0.4440
Epoch 2/4
557/557 ━━━━━━━━━━━━━━━━━━━━ 244s 439ms/step - accuracy: 0.6686 - loss: 0.4493 - val_accuracy: 0.6894 - val_loss: 0.4440
Epoch 3/4
557/557 ━━━━━━━━━━━━━━━━━━━━ 220s 395ms/step - accuracy: 0.6974 - loss: 1.7785 - val_accuracy: 0.6894 - val_loss: 0.4440
Epoch 4/4
557/557 ━━━━━━━━━━━━━━━━━━━━ 210s 377ms/step - accuracy: 0.6709 - loss: 0.4465 - val_accuracy: 0.6894 - val_loss: 0.4440
18/18 ━━━━━━━━━━━━━━━━━━━━ 9s 487ms/step - accuracy: 0.6895 - loss: 0.4360


Precisión en validación: 0.69
